In [1]:
import copy, sys, os
import numpy as np

import matplotlib.pyplot as plt

import open3d as o3d
import laspy

In [2]:
pc_path = '' #you will need to change this path
pc_fn = 'points.laz'
pc1 = laspy.read(os.path.join(pc_path, pc_fn))
xyz = np.vstack((pc1.x, pc1.y, pc1.z)).transpose()

#could be useful to subtract mean x and y position to have smaller (and easier to read) numbers
#xyz[:,0] = xyz[:,0] - xyz.mean(axis=0)[0]
#xyz[:,1] = xyz[:,1] - xyz.mean(axis=0)[1]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
print('pcd:', pcd)

pcd: PointCloud with 11480122 points.


In [3]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [4]:
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=40, std_ratio=8)
display_inlier_outlier(pcd, ind)

Showing outliers (red) and inliers (gray): 


In [6]:
o3d.io.write_point_cloud('filtered.las', pcd.select_by_index(ind))

[Open3D WARNING] Write geometry::PointCloud failed: unknown file extension las for file filtered.las.


False

In [7]:
from scipy.spatial import cKDTree

def outlier_filter(xyz, k=12, percentile=1):
    #simple outlier filter based on cKDTree
    xyz_tree = cKDTree(xyz)
    d, _ = xyz_tree.query(xyz, k=k, workers=-1)
    #2D density:
    #dens =  k / np.pi / d[:, -1]**2
    
    #3D density:
    #Volume of a sphere? 4/3 r^3 * pi
    circle_volume = 4/3 * d[:, -1]**3 * np.pi
    dens = k / circle_volume #nr of points / volume

    idx2keep, = np.where(dens > np.percentile(dens,percentile))
    xyz_tree, d = None, None
    #could also return index of files to keep idx2keep
    return xyz[idx2keep]

pt_f = outlier_filter(np.asarray(pcd.points))

pcd_f2 = o3d.geometry.PointCloud()
pcd_f2.points = o3d.utility.Vector3dVector(pt_f)

o3d.visualization.draw_geometries([pcd_f2], point_show_normal=False)